<a href="https://colab.research.google.com/github/Bmiguez/Arezzo-Valuation/blob/main/Valuation_Arezzo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importando inicialmente as bibliotecas necessárias
import pandas as pd
import numpy as np
import statsmodels.api as sm
import yfinance as yf
import sys
import datetime
from dateutil.relativedelta import relativedelta

In [ ]:
# Definindo ticker e índice de referencia
ticker = "ARZZ3.SA"
indice = "^BVSP"

# Definindo a data de referência e o intervalo de dados
ano = 2023
mes = 1
dia = 1

data_ref = datetime.date(ano, mes, dia)
data_ini = data_ref - relativedelta(months = 46)

In [ ]:
from ast import increment_lineno
## Extrair retornos ajustados da empresa e do benchmark

# Cálculo dos retornos percentuais
retornos = yf.download([ticker, indice], start = data_ini, end = data_ref, interval="1mo")["Adj Close"].pct_change()

# Remoção de valores nulos
retornos = pd.DataFrame(retornos.dropna())

y = retornos[ticker]
x = retornos[indice]

[*********************100%***********************]  2 of 2 completed


# Inicio dos cálculos
Ao realizar o valuation de uma empresa utilizando algum metódo de fluxo de caixa descontado, é necessario que haja uma taxa de desconto para que se possa trazer a valor presente os fluxos de caixa futuros da empresa. A maioria dos modelos busca estimar esta taxa através do wacc (custo médio de ponderado de capital). Uma maneira de estimar é ponderando o custo de dívida da empresa utilizando sua participação da estrutura de capital com o custo de equity da empresa, ponderando pela sua participação.

Para o custo de equity, um dos modelos mais utilizados no mercado e, o que será utilizado neste trabalho, é o CAPM. Nele calculamos o custo de equity com base em uma taxa livre de risco e no prêmio de risco de mercado ajustado para a empresa através do beta. O beta nada mais é do que um parâmetro que indica a sensibilidade da variação do preço da ação em relação aos retornos de mercado, no qual 1 indica que a ação varia junto com o mercado, valores menores que 1 indicam que a ação varia menos que o mercado e maiores que 1 variam mais que o mercado. Vale observar que valores menores que 0 indicam variação em sentidos opostos do mercado.

In [ ]:
#Iniciando pela Regressão

#Pedro: por que você começa pela regressão? Tem que explicar para que ela serve, onde vai entrar, por que você está fazendo isso.
#Pedro: começa explicando o fluxo de caixa descontado, depois WACC, depois cada componente do WACC, aí o CAPM, e aí faz a regressão.

x = sm.add_constant(x)  # Adicionando uma constante aos retornos do indice

modelo = sm.OLS(y, x)  # Gerando a regerssaão (minimos quadrados ordinários)

resultados = modelo.fit()

print(resultados.summary())

# Extraindo valor do Beta

beta = resultados.params[1]  # Primeiro valor do beta = beta estimado da regressão

conf_interval = resultados.conf_int(alpha=0.05)

lower_bound = conf_interval.iloc[1, 0]  # Retorna o limite superior do intervalo de confiança
upper_bound = conf_interval.iloc[1, 1]  # Retorna o limite inferior do intervalo de confiança

beta2 = lower_bound
beta3 = upper_bound

                            OLS Regression Results                            
Dep. Variable:               ARZZ3.SA   R-squared:                       0.551
Model:                            OLS   Adj. R-squared:                  0.541
Method:                 Least Squares   F-statistic:                     52.80
Date:                Wed, 05 Jul 2023   Prob (F-statistic):           5.30e-09
Time:                        15:48:44   Log-Likelihood:                 55.153
No. Observations:                  45   AIC:                            -106.3
Df Residuals:                      43   BIC:                            -102.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0106      0.011      0.978      0.3

In [ ]:
#Pedro: Explica no início de cada célula o que vai fazer nela.

path = "https://github.com/Bmiguez/Arezzo-Valuation/raw/main/Dados_Valuation.xlsx"
aba_bp = "BP"
aba_dre = "DRE"
aba_fc = "FC"
aba_lp = "LojasProp"
aba_indicadores = "Indicadores"
aba_receitabruta = "Receita Bruta"
aba_capex = "CAPEX"
aba_acoes = "Ações"

#### Criação dos Dataframes
#### Balanço Patrimonial
df_bp = pd.read_excel(path, aba_bp)

#### Demonstração Resultado do Exercício
df_dre = pd.read_excel(path, aba_dre)

#### Fluxo de Caixa
df_fc = pd.read_excel(path, aba_fc)

#### Lojas Próprias
df_lp = pd.read_excel(path, aba_lp)

#### Indicadores
df_ind = pd.read_excel(path, aba_indicadores)

#### Receita Bruta
df_receita = pd.read_excel(path, aba_receitabruta)

#### CAPEX
df_capex = pd.read_excel(path, aba_capex)

#### Ações
df_precos = pd.read_excel(path, aba_acoes)

Para o cálculo do custo de equity, usarei como taxa livre de risco o T-bond americano de 5 anos. Para o calculo dos retornos de mercado, o índice escolhido como proxy foi o S&P. Lembrando que prêmio de risco = beta * (rm - rf). Após o cálculo deste prêmio, como estou utilizando taxas americanas, é preciso realizar alguns ajustes. Para isso foi adicionado o risco país do Brasil e o spread entre a inflação brasileira e a inflação dos eua.

In [ ]:
## Cálculo do Custo do equity
#Pedro: antes de começar o cálculo, é importante explicar o que é custo de equity. Pode vir no comentário anterior em que peço para apresentar o modelo antes de rodar a regressão do Capm.

## Cálculo Imposto
# Definir a alíquota como o total de imposto corrente sobre o lucro antes do ir e css e aplicando a média desta % para os últimos 5 anos a partir de 2022
tax = (df_dre.loc[[8,9,10,11,12], "Corrente"] / df_dre.loc[[8,9,10,11,12], "Lucro antes do IR e CS"]).mean()

debt = -df_dre.loc[12,"Dívida total"]  # Pega o valor total da dívida em 2022
equity = df_bp.loc[12,"Patrimônio líquido"]  # Pega o valor total do Patrimonio liquido em 2022


## Risk Free Obtido T-bond 5 y 22/06
# Para o cálculo do CAPM, usarei a taxa livre de risco dos EUA (T-bond 5 years) real e a nominal
# Pedro: explique por que pega as taxas real e nominal, e como vai usar isso.
treasury_nominal = pd.read_csv("https://raw.githubusercontent.com/Bmiguez/Arezzo-Valuation/main/treasury-nominal.csv", delimiter=',', header='infer')
treasury_real = pd.read_csv("https://raw.githubusercontent.com/Bmiguez/Arezzo-Valuation/main/treasury-real.csv", delimiter=',', header='infer')

rf_usd_nominal = treasury_nominal.loc[0:29,["Date", "5 Yr"]].mean()/100 # Calcula média do último mes da taxa livre de risco nominal
rf_usd_real = treasury_real.loc[0:29,["Date", "5 YR"]].mean()/100 # Calcula média do último mes da taxa livre de risco real

# Obtendo taxas brasileiras
# Explique de onde vêm essas taxas.

ltn_br = 0.106138 # Vencimento 2026
ntnb = 0.055901 # Vencimento 2026
risco_br = 0.02402083333 # Fonte http://www.ipeadata.gov.br/ExibeSerie.aspx?serid=40940&module=M

## Inflação implicita (nominal/real)
# Razão entre os títulos nominais e os títulos reais
#Pedro: Não entendi isso. Não é definição de inflação implícita que tenho em mente, e não sei para que você vai usá-la. Reveja isso, e explica com cuidado.

cpi_usd = (1 + rf_usd_nominal[0])/(1 + rf_usd_real[0])
cpi_br = (1 + ltn_br)/(1 + ntnb)


## Calculando Retorno de mercado
data_ini = data_ref - relativedelta(years = 5)

retornos = yf.download("^GSPC", start = data_ini, end = data_ref, interval="1mo")["Adj Close"].pct_change().dropna() # Obtendo os retornos percentuais do S&P

retorno_anualizado = (float(retornos.mean())+1)**12-1

## Equity Risk Premium
# Calculo do ke para cada beta
# O ke é o custo do equity e é utilizado para o calculo do wacc (média ponderada entre o custo do equity e o custo da dívida)
#Pedro: defina Ke, apresente e explique a fórmula, e diga para que você precisa disso.
ERP = (retorno_anualizado - rf_usd_nominal.iloc[0])
ke1 = rf_usd_nominal[0] + beta * ERP + (cpi_br/cpi_usd -1) + risco_br
ke2 = rf_usd_nominal[0] + beta2 * ERP + (cpi_br/cpi_usd -1) + risco_br
ke3 = rf_usd_nominal[0] + beta3 * ERP + (cpi_br/cpi_usd -1) + risco_br

# Custo de dívida kd
# Como dito anteriormente, o custo da dívida é um dos componentes do wacc. Este representa quanto em média a empresa paga de juros para a sua dívida
# Pedro: explique aqui também.
resultado_fin2022 = df_dre.loc[12,"Resultado Financeiro"]

divida = df_bp.loc[[11,12],["Empréstimos e financiamentos","Empréstimos e financiamentos.1"]].mean(axis=0).sum(axis=0)

kd = (-resultado_fin2022/divida) * (1-tax) # Sinal de - no resultado financiero pois valores na planilha estão negativos
# Pedro: por que o sinal de menos? Imagino que o resultado financeiro esteja apresentado com valores negativos.
## Calculando o WACC

wacc_1 = ke1 * (equity/(debt+equity)) + kd * (debt/(debt+equity))
wacc_2 = ke2 * (equity/(debt+equity)) + kd * (debt/(debt+equity))
wacc_3 = ke3 * (equity/(debt+equity)) + kd * (debt/(debt+equity))

## Obtendo o número de ações

acoes = 110095459/1000 # Receitas e custos estão divididos por 1000
#Pedro: diga de onde vem esse número. Evite imputar números à mão - o ideal é puxar de uma fonte bem identificada. No mínimo, indique claramente onde encontrar esse número. É fundamental garantir replicabilidade.

[*********************100%***********************]  1 of 1 completed


<ipython-input-17-debc3312f3e3>:18: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  rf_usd_nominal = treasury_nominal.loc[0:29,["Date", "5 Yr"]].mean()/100 # Calcula média do último mes da taxa livre de risco nominal
<ipython-input-17-debc3312f3e3>:19: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  rf_usd_real = treasury_real.loc[0:29,["Date", "5 YR"]].mean()/100 # Calcula média do último mes da taxa livre de risco real


In [ ]:
### Calculando a média das deduções entre receita bruta e liquida
receita_bruta = df_receita.iloc[:,1]
receita_liquida = df_dre.iloc[:,1]

deducoes = pd.DataFrame({"Ano": range(2010,2023), "Percentual":receita_liquida/receita_bruta})

#Pedro: explique a relação entre 'média de deduções' e 'razão entre receita líquida e receita bruta'.
# A relação entre receita líquida e receita bruta se mantem relativamente constante com o tempo. Entre as
# duas contas, se encontram devoluções e impostos sobre venda.

## Calculando a média das deduções nos de 2022,2019,2018,2017,2016
multiplicador_receita_liq = deducoes.loc[[12,9,8,7,6],"Percentual"].mean()

### Organizando projeções

## Número de lojas

total_FR = df_lp.loc[1:,"Número de franquias"].reset_index(drop=True) # Numero de lojas franquias por ano
total_LP = df_lp.loc[1:,"Número de lojas próprias"].reset_index(drop=True) # Numero de lojas proprias por ano
total_mult = df_lp.loc[1:,"Multimarca MI (sem overlap)"].reset_index(drop=True) # Numero de lojas multimarcas por ano

# Calculando o numero de aberturas por ano (diferença entre o ano e o ano anterior)
abert_FR = df_lp.loc[:,"Número de franquias"].diff().dropna().reset_index(drop=True)
abert_LP = df_lp.loc[:,"Número de lojas próprias"].diff().dropna().reset_index(drop=True)
abert_mult = df_lp.loc[:,"Multimarca MI (sem overlap)"].diff().dropna().reset_index(drop=True)

## Receita por canal
canais = ["Franquias", "Multimarcas", "Lojas próprias", "Web Commerce", "Outros²"]
df_canais = df_receita.loc[:,canais]

receita_LP = df_canais.loc[:,"Lojas próprias"] # Separa o total da receita de lojas proprias por ano
receita_FR = df_canais.loc[:,"Franquias"] # Separa o total da receita de franquias por ano
receita_mult = df_canais.loc[:,"Multimarcas"] # Separa o total da receita de multimarcas por ano
receita_web = df_canais.loc[:,"Web Commerce"] # Separa o total da receita de ecommerce por ano
receita_out = df_canais.loc[:,"Outros²"] # Separa o total de outras receitas por ano
receita_ext = df_receita.loc[:,"Mercado externo"] # Separa o total da receita de exportação por ano

## Receita por loja

receita_por_LP = receita_LP/ total_LP # Receita por loja própria
receita_por_FR = receita_FR / total_FR # Receita por franquia
receita_por_mult = receita_mult / total_mult # Receita por multimarca

## Crescimento receita por unidade + receita e-commerec (variação percentual ano a ano)
# LP = lojas próprias, FR = franquias, mult = multimarcas, web = e-commerce e ext = exportações
#Pedro: o que é 'econ'? Defina todas as siglas.
var_por_LP = receita_por_LP.pct_change()
var_por_FR = receita_por_FR.pct_change()
var_por_mult = receita_por_mult.pct_change()
var_web = receita_web.pct_change()
var_ext = receita_ext.pct_change()

## Projeções de crescimento de receita
# Utilizando a média de crescimento (em percentual) de 2022, 2019, 2018, 2017, 2016
#Pedro: não entendi bem a fórmula. Como sempre, é importante apresentar a fórmula antes do código, e explicá-la.
proj_rec_LP = (var_por_LP[12] + var_por_LP[9] + var_por_LP[8] + var_por_LP[7] + var_por_LP[6])/5 + 1
proj_rec_FR = (var_por_FR[12] + var_por_FR[9] + var_por_FR[8] + var_por_FR[7] + var_por_FR[6])/5 + 1
proj_rec_mult = (var_por_mult[12] + var_por_mult[9] + var_por_mult[8] + var_por_mult[7] + var_por_mult[6])/5 + 1
proj_rec_out = (receita_out[12] + receita_out[9] + receita_out[8] + receita_out[7] + receita_out[6])/5 + 1
proj_rec_web = (var_web[12] + var_web[9] + var_web[8] + var_web[7] + var_web[6])/5 + 1
proj_rec_ext = (var_ext[12] + var_ext[9] + var_ext[8] + var_ext[7] + var_ext[6])/5 + 1

# Projeções de número de unidades
# Utilizando a média do numero de aberturas de 2022, 2019, 2018, 2017, 2016
proj_abert_FR = (abert_FR[12] + abert_FR[9] + abert_FR[8] + abert_FR[7] + abert_FR[6])/5
proj_abert_LP = (abert_LP[12] + abert_LP[9] + abert_LP[8] + abert_LP[7] + abert_LP[6])/5
proj_abert_mult = (abert_mult[12] + abert_mult[9] + abert_mult[8] + abert_mult[7] + abert_mult[6])/5

### 1ª Valuation

2015-2022


Estimados 2023 - 2027


Beta = valor estimado da regressão

In [ ]:
### Primeiro valuation - 2015 até 2022 (ano 2022)
## Criando uma lista para armazenar os valores de cada valuation
lista_precos = []
lista_anos = []

## Construção do for loop: vai trocando o ano no qual as projeções são feitas através do indice i
#Pedro: apresente a fórmula antes de entrar no código para cada bloco abaixo.

for i in [5,6,7,8,9,10,11,12]:

    ano = i + 2010
    ## Projetando receita bruta (Nomencaltura: 1 = primeiro ano projetado)

    # Receita por Loja própria = Receita do ano anterior * (1 + crescimento médio projetado)
    receita_lp_1 = receita_por_LP[i] * proj_rec_LP
    receita_lp_2 = receita_lp_1 * proj_rec_LP
    receita_lp_3 = receita_lp_2 * proj_rec_LP
    receita_lp_4 = receita_lp_3 * proj_rec_LP
    receita_lp_5 = receita_lp_4 * proj_rec_LP

    # Total Lojas própriras = Numero de lojas do ano anterior + projeção de aberturas
    total_LP_1 = total_LP[i] + proj_abert_LP
    total_LP_2 = total_LP_1 + proj_abert_LP
    total_LP_3 = total_LP_2 + proj_abert_LP
    total_LP_4 = total_LP_3 + proj_abert_LP
    total_LP_5 = total_LP_4 + proj_abert_LP

    # Receita por Franquia = Receita do ano anterior * (1 + crescimento médio projetado)
    receita_fr_1 = receita_por_FR[i] * proj_rec_FR
    receita_fr_2 = receita_fr_1 * proj_rec_FR
    receita_fr_3 = receita_fr_2 * proj_rec_FR
    receita_fr_4 = receita_fr_3 * proj_rec_FR
    receita_fr_5 = receita_fr_4 * proj_rec_FR

    # Total Franquias = Numero de lojas do ano anterior + projeção de aberturas
    total_FR_1 =  total_FR[i] + proj_abert_FR
    total_FR_2 = total_FR_1 + proj_abert_FR
    total_FR_3 = total_FR_2 + proj_abert_FR
    total_FR_4 = total_FR_3 + proj_abert_FR
    total_FR_5 = total_FR_4 + proj_abert_FR

    # Receita por multimarca = Receita do ano anterior * (1 + crescimento médio projetado)
    receita_mult_1 = receita_por_mult[i] * proj_rec_mult
    receita_mult_2 = receita_mult_1 * proj_rec_mult
    receita_mult_3 = receita_mult_2 * proj_rec_mult
    receita_mult_4 = receita_mult_3 * proj_rec_mult
    receita_mult_5 = receita_mult_4 * proj_rec_mult

    # Total Multimarcas = Numero de lojas do ano anterior + projeção de aberturas
    total_mult_1 =  total_mult[12] + proj_abert_mult #Projetando do ano base
    total_mult_2 = total_mult_1 + proj_abert_mult
    total_mult_3 = total_mult_2 + proj_abert_mult
    total_mult_4 = total_mult_3 + proj_abert_mult
    total_mult_5 = total_mult_4 + proj_abert_mult

    # Faturamento Web commerce = Receita do ano anterior * (1 + crescimento médio projetado)
    receita_web_1 = receita_web[i] * proj_rec_web
    receita_web_2 = receita_web_1 * (proj_rec_web - 0.025) #Suavizando a estimativa de crescimento para os anos seguintes
    receita_web_3 = receita_web_2 * (proj_rec_web - 0.030)
    receita_web_4 = receita_web_3 * (proj_rec_web - 0.050)
    receita_web_5 = receita_web_4 * (proj_rec_web - 0.100)

    # Faturamento Outros = Média da receita dos anos considerados

    receita_out_1 = proj_rec_out
    receita_out_2 = proj_rec_out
    receita_out_3 = proj_rec_out
    receita_out_4 = proj_rec_out
    receita_out_5 = proj_rec_out

    # Faturamento Exterior = Receita do ano anterior * (1 + crescimento médio projetado)
    receita_ext_1 = receita_ext[i] * proj_rec_ext #Projetando do ano base
    receita_ext_2 = receita_ext_1 * proj_rec_ext
    receita_ext_3 = receita_ext_2 * proj_rec_ext
    receita_ext_4 = receita_ext_3 * proj_rec_ext
    receita_ext_5 = receita_ext_4 * proj_rec_ext

    ## Cálculo da Receita Bruta total

    receita_bruta_1 = (receita_lp_1 * total_LP_1) + (receita_fr_1 * total_FR_1) + (receita_mult_1 * total_mult_1) + receita_web_1 + receita_out_1 + receita_ext_1
    receita_bruta_2 = (receita_lp_2 * total_LP_2) + (receita_fr_2 * total_FR_2) + (receita_mult_2 * total_mult_2) + receita_web_2 + receita_out_2 + receita_ext_2
    receita_bruta_3 = (receita_lp_3 * total_LP_3) + (receita_fr_3 * total_FR_3) + (receita_mult_3 * total_mult_3) + receita_web_3 + receita_out_3 + receita_ext_3
    receita_bruta_4 = (receita_lp_4 * total_LP_4) + (receita_fr_4 * total_FR_4) + (receita_mult_4 * total_mult_4) + receita_web_4 + receita_out_4 + receita_ext_4
    receita_bruta_5 = (receita_lp_5 * total_LP_5) + (receita_fr_5 * total_FR_5) + (receita_mult_5 * total_mult_5) + receita_web_5 + receita_out_5 + receita_ext_5

    receita_projetada = pd.DataFrame({"Ano": [ano + 1, ano + 2, ano + 3, ano + 4, ano + 5], "Receita Bruta":[receita_bruta_1, receita_bruta_2, receita_bruta_3, receita_bruta_4, receita_bruta_5]})
    receita_projetada["Receita Liquida"] = receita_projetada["Receita Bruta"] * multiplicador_receita_liq


    ### Cálculo do Lucro Bruto
    ## Markups
    #Pedro: explique o que é e para que você vai usar markup.
    df_cmv = df_dre["CMV"]

    # Markup = razão da receita bruta / CMV
    markup = receita_bruta / (df_cmv * -1)
    var_markup = markup.pct_change()
    cresc_markup = var_markup.mean() + 1

    # Projeta o crescimento do markup com base nos últimos anos (mudança na margem bruta)
    markup_1 = markup[i] * cresc_markup
    markup_2 = markup_1 * cresc_markup
    markup_3 = markup_2 * cresc_markup
    markup_4 = markup_3 * cresc_markup
    markup_5 = markup_4 * cresc_markup

    receita_projetada["Markups"] = [markup_1, markup_2, markup_3, markup_4, markup_5]
    receita_projetada["CMV"] = receita_projetada["Receita Liquida"] / (receita_projetada["Markups"] * -1)

    receita_projetada["Lucro Bruto"] = receita_projetada["Receita Liquida"] + receita_projetada["CMV"]


    ### Projeções de Despesas
    ## Despesas de Lojas próprias e Web Commerce (percentual da receita de lojas proprias + web commerce)

    desp_lojasecom = df_dre.loc[[6,7,8,9,12],"Lojas próprias e Web Commerce"].sum(axis=0)
    particip_desp_lojasecom = desp_lojasecom / (receita_LP.loc[[6,7,8,9,12]] + receita_web.loc[[6,7,8,9,12]]).sum(axis=0)

    ## Projetando Despesas de Lojas próprias e Web Commerce
#Pedro: explique a fórmula aqui também. Esse é um comentário geral, vale para o código todo - não vou repetir em cada bloco, mas é importante rever o código para implementar esse ajuste de ponta a ponta.
    desp_lojasecom_1 = (receita_web_1 + receita_lp_1) * particip_desp_lojasecom
    desp_lojasecom_2 = (receita_web_2 + receita_lp_2) * particip_desp_lojasecom
    desp_lojasecom_3 = (receita_web_3 + receita_lp_3) * particip_desp_lojasecom
    desp_lojasecom_4 = (receita_web_4 + receita_lp_4) * particip_desp_lojasecom
    desp_lojasecom_5 = (receita_web_5 + receita_lp_5) * particip_desp_lojasecom

    ## Despesas de venda, logística e suprimentos (percentual da receita total)

    desp_venda = df_dre.loc[[6,7,8,9,12],"Venda, logística e suprimentos"].sum(axis=0)

    particip_desp_venda = desp_venda / df_receita.loc[[6,7,8,9,12],"Receita bruta total"].sum(axis=0)

    ## Projetando Despesas de venda, logística e suprimentos

    desp_venda_1 = receita_bruta_1 * particip_desp_venda
    desp_venda_2 = receita_bruta_2 * particip_desp_venda
    desp_venda_3 = receita_bruta_3 * particip_desp_venda
    desp_venda_4 = receita_bruta_4 * particip_desp_venda
    desp_venda_5 = receita_bruta_5 * particip_desp_venda

    ## G&A (Premissa: Despesas de G&A não tem uma correlação com receita, portanto vão crescer inflação + 5%)
    # Pedro: defina todas as siglas. Explique de onde vem esse 5%.
    desp_gea = df_dre.loc[:,"Despesas gerais e administrativas"]
    desp_gea_1 = desp_gea[i] * (cpi_br + 0.05)
    desp_gea_2 = desp_gea_1 * (cpi_br + 0.05)
    desp_gea_3 = desp_gea_2 * (cpi_br + 0.05)
    desp_gea_4 = desp_gea_3 * (cpi_br + 0.05)
    desp_gea_5 = desp_gea_4 * (cpi_br + 0.05)

    ## Outras despesas (média de outras despeas dos anos 2022, 2021, 2018, 2017, 2016)
    desp_outros = df_dre.loc[:, "Outras (despesas) e receitas"]
    media_outros = desp_outros.iloc[[6,7,8,11,12]].mean(axis=0) #2019 outlier
    # Pedro: explique o outlier com clareza no texto da monografia.

    ## Criando dataframe para armazenar todas as despesas
    despesas = pd.DataFrame({"Despesas LP e Ecom": [desp_lojasecom_1, desp_lojasecom_2, desp_lojasecom_3, desp_lojasecom_4, desp_lojasecom_5],
                             "Despesas Venda e Logística": [desp_venda_1, desp_venda_2, desp_venda_3, desp_venda_4, desp_venda_5 ],
                             "Despesas G&A": [desp_gea_1, desp_gea_2, desp_gea_3, desp_gea_4, desp_gea_5],
                            "Outras despesas": [media_outros, media_outros, media_outros, media_outros, media_outros]})

    ## Cálculo EBITDA (lucro bruto + despeas)
    # Corrija 'despeas' e defina a sigla EBITDA.
    ebitda =  receita_projetada["Lucro Bruto"] + despesas.sum(axis=1)

    ## Impostos
    #Pedro: diga que impostos são esses.
    particip_impostos = df_dre.loc[[8,9,10,11,12],"Corrente"] / df_dre.loc[[8,9,10,11,12],"EBITDA"]# Participação dos impostos sobre o EBITDA
    media_impostos = particip_impostos.mean()

    impostos = ebitda * media_impostos

    ## CAPEX (percentual do capex sobre o ebitda)
    #Pedro: defina a sigla.
    media_capex = df_capex.loc[[8,9,10,11,12],"CAPEX total"] / df_dre.loc[[8,9,10,11,12],"EBITDA"]
    media_capex = media_capex.mean()

    capex = ebitda * media_capex

    ## Capital de giro
    # Capital de giro calculado com dados extraidos da aba bp
    cg = df_bp.loc[:,["Ativo circulante","Caixa e equivalentes de caixa","Aplicações financeiras", "Passivo circulante","Empréstimos e financiamentos"]]
    cg = (cg["Ativo circulante"] - cg["Caixa e equivalentes de caixa"] - cg["Aplicações financeiras"]) - (cg["Passivo circulante"] - cg["Empréstimos e financiamentos"])
    particip_cg = cg.iloc[[10,11,12]] / df_dre.loc[[10,11,12],"Receita Líquida"]
    particip_cg = particip_cg.mean()

    # Projeção realizada com a participação do capital de giro na receita líquida
    cg_proj = receita_projetada.loc[:, "Receita Liquida"] * particip_cg

    # Cálculo da variação de cg
    var_cg = cg_proj.diff()
    var_cg[0] = cg_proj[1]- cg[i]

    ## FCE
    fce = ebitda - impostos - capex + var_cg

    ## Taxas de desconto

    fator_desconto = [1 + wacc_1, (1 + wacc_1)**2, (1 + wacc_1)**3, (1 + wacc_1)**4, (1 + wacc_1)**5]

    fce_descontado = fce/fator_desconto

    ## Perpetuidade

    g = 0.05 #Pedro: explique esse número.

    perpetuidade = fce[4]*((1+g)/(wacc_1-g))
    vp_perpetuidade = perpetuidade / (1 + wacc_1)**5

    ## Preço justo
    caixaliq = df_dre.loc[i,"Caixa líquido³"]
    valor_eco = fce_descontado.sum(axis=0) + vp_perpetuidade + caixaliq
    preco = valor_eco / acoes

    lista_precos.append(preco)
    lista_anos.append(ano)

resultado_val_1 = pd.DataFrame({"Ano": lista_anos,"Preço": lista_precos})
print(resultado_val_1)


    Ano       Preço
0  2015   91.953393
1  2016   98.854344
2  2017  102.992852
3  2018  107.505327
4  2019  112.396404
5  2020   90.932797
6  2021  150.534996
7  2022  221.095721


### 2ª Valuation

Pedro: parei de rever aqui; faça na sequência os mesmos ajustes que pedi acima. Suponho que a estrutura seja igual.

Gráfico 2015-2022


Estimados 2023 - 2027


Beta = limite inferior do intervalo de confiança

In [ ]:
### Segundo valuation - 2015 até 2022

lista_precos_2 = []
lista_anos = []
## Construção do for loop: vai trocando o ano no qual as projeções são feitas através do indice i

for i in [5,6,7,8,9,10,11,12]:

    ano = i + 2010
    ## Projetando receita bruta (Nomencaltura: 1 = primeiro ano projetado)

    # Receita por Loja própria = Receita do ano anterior * (1 + crescimento médio projetado)
    receita_lp_1 = receita_por_LP[i] * proj_rec_LP
    receita_lp_2 = receita_lp_1 * proj_rec_LP
    receita_lp_3 = receita_lp_2 * proj_rec_LP
    receita_lp_4 = receita_lp_3 * proj_rec_LP
    receita_lp_5 = receita_lp_4 * proj_rec_LP

    # Total Lojas própriras = Numero de lojas do ano anterior + projeção de aberturas
    total_LP_1 = total_LP[i] + proj_abert_LP
    total_LP_2 = total_LP_1 + proj_abert_LP
    total_LP_3 = total_LP_2 + proj_abert_LP
    total_LP_4 = total_LP_3 + proj_abert_LP
    total_LP_5 = total_LP_4 + proj_abert_LP

    # Receita por Franquia = Receita do ano anterior * (1 + crescimento médio projetado)
    receita_fr_1 = receita_por_FR[i] * proj_rec_FR
    receita_fr_2 = receita_fr_1 * proj_rec_FR
    receita_fr_3 = receita_fr_2 * proj_rec_FR
    receita_fr_4 = receita_fr_3 * proj_rec_FR
    receita_fr_5 = receita_fr_4 * proj_rec_FR

    # Total Franquias = Numero de lojas do ano anterior + projeção de aberturas
    total_FR_1 =  total_FR[i] + proj_abert_FR
    total_FR_2 = total_FR_1 + proj_abert_FR
    total_FR_3 = total_FR_2 + proj_abert_FR
    total_FR_4 = total_FR_3 + proj_abert_FR
    total_FR_5 = total_FR_4 + proj_abert_FR

    # Receita por multimarca = Receita do ano anterior * (1 + crescimento médio projetado)
    receita_mult_1 = receita_por_mult[i] * proj_rec_mult
    receita_mult_2 = receita_mult_1 * proj_rec_mult
    receita_mult_3 = receita_mult_2 * proj_rec_mult
    receita_mult_4 = receita_mult_3 * proj_rec_mult
    receita_mult_5 = receita_mult_4 * proj_rec_mult

    # Total Multimarcas = Numero de lojas do ano anterior + projeção de aberturas
    total_mult_1 =  total_mult[12] + proj_abert_mult #Projetando do ano base
    total_mult_2 = total_mult_1 + proj_abert_mult
    total_mult_3 = total_mult_2 + proj_abert_mult
    total_mult_4 = total_mult_3 + proj_abert_mult
    total_mult_5 = total_mult_4 + proj_abert_mult

    # Faturamento Web commerce = Receita do ano anterior * (1 + crescimento médio projetado)
    receita_web_1 = receita_web[i] * proj_rec_web
    receita_web_2 = receita_web_1 * (proj_rec_web - 0.025) #Suavizando a estimativa de crescimento para os anos seguintes
    receita_web_3 = receita_web_2 * (proj_rec_web - 0.030)
    receita_web_4 = receita_web_3 * (proj_rec_web - 0.050)
    receita_web_5 = receita_web_4 * (proj_rec_web - 0.100)

    # Faturamento Outros = Média da receita dos anos considerados

    receita_out_1 = proj_rec_out
    receita_out_2 = proj_rec_out
    receita_out_3 = proj_rec_out
    receita_out_4 = proj_rec_out
    receita_out_5 = proj_rec_out

    # Faturamento Exterior = Receita do ano anterior * (1 + crescimento médio projetado)
    receita_ext_1 = receita_ext[i] * proj_rec_ext #Projetando do ano base
    receita_ext_2 = receita_ext_1 * proj_rec_ext
    receita_ext_3 = receita_ext_2 * proj_rec_ext
    receita_ext_4 = receita_ext_3 * proj_rec_ext
    receita_ext_5 = receita_ext_4 * proj_rec_ext

    ## Cálculo da Receita Bruta total

    receita_bruta_1 = (receita_lp_1 * total_LP_1) + (receita_fr_1 * total_FR_1) + (receita_mult_1 * total_mult_1) + receita_web_1 + receita_out_1 + receita_ext_1
    receita_bruta_2 = (receita_lp_2 * total_LP_2) + (receita_fr_2 * total_FR_2) + (receita_mult_2 * total_mult_2) + receita_web_2 + receita_out_2 + receita_ext_2
    receita_bruta_3 = (receita_lp_3 * total_LP_3) + (receita_fr_3 * total_FR_3) + (receita_mult_3 * total_mult_3) + receita_web_3 + receita_out_3 + receita_ext_3
    receita_bruta_4 = (receita_lp_4 * total_LP_4) + (receita_fr_4 * total_FR_4) + (receita_mult_4 * total_mult_4) + receita_web_4 + receita_out_4 + receita_ext_4
    receita_bruta_5 = (receita_lp_5 * total_LP_5) + (receita_fr_5 * total_FR_5) + (receita_mult_5 * total_mult_5) + receita_web_5 + receita_out_5 + receita_ext_5

    receita_projetada = pd.DataFrame({"Ano": [ano + 1, ano + 2, ano + 3, ano + 4, ano + 5], "Receita Bruta":[receita_bruta_1, receita_bruta_2, receita_bruta_3, receita_bruta_4, receita_bruta_5]})
    receita_projetada["Receita Liquida"] = receita_projetada["Receita Bruta"] * multiplicador_receita_liq


    ### Cálculo do Lucro Bruto
    ## Markups
    df_cmv = df_dre["CMV"]

    # Markup = razão da receita bruta / CMV
    markup = receita_bruta / (df_cmv * -1)
    var_markup = markup.pct_change()
    cresc_markup = var_markup.mean() + 1

    # Projeta o crescimento do markup com base nos últimos anos (mudança na margem bruta)
    markup_1 = markup[i] * cresc_markup
    markup_2 = markup_1 * cresc_markup
    markup_3 = markup_2 * cresc_markup
    markup_4 = markup_3 * cresc_markup
    markup_5 = markup_4 * cresc_markup

    receita_projetada["Markups"] = [markup_1, markup_2, markup_3, markup_4, markup_5]
    receita_projetada["CMV"] = receita_projetada["Receita Liquida"] / (receita_projetada["Markups"] * -1)

    receita_projetada["Lucro Bruto"] = receita_projetada["Receita Liquida"] + receita_projetada["CMV"]


    ### Projeções de Despesas
    ## Despesas de Lojas próprias e Web Commerce (percentual da receita de lojas proprias + web commerce)

    desp_lojasecom = df_dre.loc[[6,7,8,9,12],"Lojas próprias e Web Commerce"].sum(axis=0)
    particip_desp_lojasecom = desp_lojasecom / (receita_LP.loc[[6,7,8,9,12]] + receita_web.loc[[6,7,8,9,12]]).sum(axis=0)

    ## Projetando Despesas de Lojas próprias e Web Commerce

    desp_lojasecom_1 = (receita_web_1 + receita_lp_1) * particip_desp_lojasecom
    desp_lojasecom_2 = (receita_web_2 + receita_lp_2) * particip_desp_lojasecom
    desp_lojasecom_3 = (receita_web_3 + receita_lp_3) * particip_desp_lojasecom
    desp_lojasecom_4 = (receita_web_4 + receita_lp_4) * particip_desp_lojasecom
    desp_lojasecom_5 = (receita_web_5 + receita_lp_5) * particip_desp_lojasecom

    ## Despesas de venda, logística e suprimentos (percentual da receita total)

    desp_venda = df_dre.loc[[6,7,8,9,12],"Venda, logística e suprimentos"].sum(axis=0)

    particip_desp_venda = desp_venda / df_receita.loc[[6,7,8,9,12],"Receita bruta total"].sum(axis=0)

    ## Projetando Despesas de venda, logística e suprimentos

    desp_venda_1 = receita_bruta_1 * particip_desp_venda
    desp_venda_2 = receita_bruta_2 * particip_desp_venda
    desp_venda_3 = receita_bruta_3 * particip_desp_venda
    desp_venda_4 = receita_bruta_4 * particip_desp_venda
    desp_venda_5 = receita_bruta_5 * particip_desp_venda

    ## G&A (Premissa: Despesas de G&A não tem uma correlação com receita, portanto vão crescer inflação + 5%)
    desp_gea = df_dre.loc[:,"Despesas gerais e administrativas"]
    desp_gea_1 = desp_gea[i] * (cpi_br + 0.05)
    desp_gea_2 = desp_gea_1 * (cpi_br + 0.05)
    desp_gea_3 = desp_gea_2 * (cpi_br + 0.05)
    desp_gea_4 = desp_gea_3 * (cpi_br + 0.05)
    desp_gea_5 = desp_gea_4 * (cpi_br + 0.05)

    ## Outras despesas (média de outras despeas dos anos 2022, 2021, 2018, 2017, 2016)
    desp_outros = df_dre.loc[:, "Outras (despesas) e receitas"]
    media_outros = desp_outros.iloc[[6,7,8,11,12]].mean(axis=0) #2019 outlier


    ## Criando dataframe para armazenar todas as despesas
    despesas = pd.DataFrame({"Despesas LP e Ecom": [desp_lojasecom_1, desp_lojasecom_2, desp_lojasecom_3, desp_lojasecom_4, desp_lojasecom_5],
                             "Despesas Venda e Logística": [desp_venda_1, desp_venda_2, desp_venda_3, desp_venda_4, desp_venda_5 ],
                             "Despesas G&A": [desp_gea_1, desp_gea_2, desp_gea_3, desp_gea_4, desp_gea_5],
                            "Outras despesas": [media_outros, media_outros, media_outros, media_outros, media_outros]})

    ## Cálculo EBITDA (lucro bruto + despeas)

    ebitda =  receita_projetada["Lucro Bruto"] + despesas.sum(axis=1)

    ## Impostos
    particip_impostos = df_dre.loc[[8,9,10,11,12],"Corrente"] / df_dre.loc[[8,9,10,11,12],"EBITDA"]# Participação dos impostos sobre o EBITDA
    media_impostos = particip_impostos.mean()

    impostos = ebitda * media_impostos

    ## CAPEX (percentual do capex sobre o ebitda)
    media_capex = df_capex.loc[[8,9,10,11,12],"CAPEX total"] / df_dre.loc[[8,9,10,11,12],"EBITDA"]
    media_capex = media_capex.mean()

    capex = ebitda * media_capex

    ## Capital de giro
    # Capital de giro calculado com dados extraidos da aba bp
    cg = df_bp.loc[:,["Ativo circulante","Caixa e equivalentes de caixa","Aplicações financeiras", "Passivo circulante","Empréstimos e financiamentos"]]
    cg = (cg["Ativo circulante"] - cg["Caixa e equivalentes de caixa"] - cg["Aplicações financeiras"]) - (cg["Passivo circulante"] - cg["Empréstimos e financiamentos"])
    particip_cg = cg.iloc[[10,11,12]] / df_dre.loc[[10,11,12],"Receita Líquida"]
    particip_cg = particip_cg.mean()

    # Projeção realizada com a participação do capital de giro na receita líquida
    cg_proj = receita_projetada.loc[:, "Receita Liquida"] * particip_cg

    # Cálculo da variação de cg
    var_cg = cg_proj.diff()
    var_cg[0] = cg_proj[1]- cg[i]

    ## FCE
    fce = ebitda - impostos - capex + var_cg

    ## Taxas de desconto

    fator_desconto = [1 + wacc_2, (1 + wacc_2)**2, (1 + wacc_2)**3, (1 + wacc_2)**4, (1 + wacc_2)**5]

    fce_descontado = fce/fator_desconto

    ## Perpetuidade

    g = 0.05

    perpetuidade = fce[4]*((1+g)/(wacc_2-g))
    vp_perpetuidade = perpetuidade / (1 + wacc_2)**5

    ## Preço justo
    caixaliq = df_dre.loc[i,"Caixa líquido³"]
    valor_eco = fce_descontado.sum(axis=0) + vp_perpetuidade + caixaliq
    preco = valor_eco / acoes

    lista_precos_2.append(preco)
    lista_anos.append(ano)
resultado_val_2 = pd.DataFrame({"Ano": lista_anos,"Preço": lista_precos_2})
print(resultado_val_2)

    Ano       Preço
0  2015  105.289922
1  2016  113.156238
2  2017  117.828920
3  2018  123.106902
4  2019  128.753378
5  2020  104.423587
6  2021  172.932895
7  2022  253.469632


### 3ª Valuation

Gráfico 2015-2022


Estimados 2023 - 2027


Beta = limite superior do intervalo de confiança

In [ ]:
### Segundo valuation - 2015 até 2022

lista_precos_3 = []
lista_anos = []
## Construção do for loop: vai trocando o ano no qual as projeções são feitas através do indice i

for i in [5,6,7,8,9,10,11,12]:

    ano = i + 2010
    ## Projetando receita bruta (Nomencaltura: 1 = primeiro ano projetado)

    # Receita por Loja própria = Receita do ano anterior * (1 + crescimento médio projetado)
    receita_lp_1 = receita_por_LP[i] * proj_rec_LP
    receita_lp_2 = receita_lp_1 * proj_rec_LP
    receita_lp_3 = receita_lp_2 * proj_rec_LP
    receita_lp_4 = receita_lp_3 * proj_rec_LP
    receita_lp_5 = receita_lp_4 * proj_rec_LP

    # Total Lojas própriras = Numero de lojas do ano anterior + projeção de aberturas
    total_LP_1 = total_LP[i] + proj_abert_LP
    total_LP_2 = total_LP_1 + proj_abert_LP
    total_LP_3 = total_LP_2 + proj_abert_LP
    total_LP_4 = total_LP_3 + proj_abert_LP
    total_LP_5 = total_LP_4 + proj_abert_LP

    # Receita por Franquia = Receita do ano anterior * (1 + crescimento médio projetado)
    receita_fr_1 = receita_por_FR[i] * proj_rec_FR
    receita_fr_2 = receita_fr_1 * proj_rec_FR
    receita_fr_3 = receita_fr_2 * proj_rec_FR
    receita_fr_4 = receita_fr_3 * proj_rec_FR
    receita_fr_5 = receita_fr_4 * proj_rec_FR

    # Total Franquias = Numero de lojas do ano anterior + projeção de aberturas
    total_FR_1 =  total_FR[i] + proj_abert_FR
    total_FR_2 = total_FR_1 + proj_abert_FR
    total_FR_3 = total_FR_2 + proj_abert_FR
    total_FR_4 = total_FR_3 + proj_abert_FR
    total_FR_5 = total_FR_4 + proj_abert_FR

    # Receita por multimarca = Receita do ano anterior * (1 + crescimento médio projetado)
    receita_mult_1 = receita_por_mult[i] * proj_rec_mult
    receita_mult_2 = receita_mult_1 * proj_rec_mult
    receita_mult_3 = receita_mult_2 * proj_rec_mult
    receita_mult_4 = receita_mult_3 * proj_rec_mult
    receita_mult_5 = receita_mult_4 * proj_rec_mult

    # Total Multimarcas = Numero de lojas do ano anterior + projeção de aberturas
    total_mult_1 =  total_mult[12] + proj_abert_mult #Projetando do ano base
    total_mult_2 = total_mult_1 + proj_abert_mult
    total_mult_3 = total_mult_2 + proj_abert_mult
    total_mult_4 = total_mult_3 + proj_abert_mult
    total_mult_5 = total_mult_4 + proj_abert_mult

    # Faturamento Web commerce = Receita do ano anterior * (1 + crescimento médio projetado)
    receita_web_1 = receita_web[i] * proj_rec_web
    receita_web_2 = receita_web_1 * (proj_rec_web - 0.025) #Suavizando a estimativa de crescimento para os anos seguintes
    receita_web_3 = receita_web_2 * (proj_rec_web - 0.030)
    receita_web_4 = receita_web_3 * (proj_rec_web - 0.050)
    receita_web_5 = receita_web_4 * (proj_rec_web - 0.100)

    # Faturamento Outros = Média da receita dos anos considerados

    receita_out_1 = proj_rec_out
    receita_out_2 = proj_rec_out
    receita_out_3 = proj_rec_out
    receita_out_4 = proj_rec_out
    receita_out_5 = proj_rec_out

    # Faturamento Exterior = Receita do ano anterior * (1 + crescimento médio projetado)
    receita_ext_1 = receita_ext[i] * proj_rec_ext #Projetando do ano base
    receita_ext_2 = receita_ext_1 * proj_rec_ext
    receita_ext_3 = receita_ext_2 * proj_rec_ext
    receita_ext_4 = receita_ext_3 * proj_rec_ext
    receita_ext_5 = receita_ext_4 * proj_rec_ext

    ## Cálculo da Receita Bruta total

    receita_bruta_1 = (receita_lp_1 * total_LP_1) + (receita_fr_1 * total_FR_1) + (receita_mult_1 * total_mult_1) + receita_web_1 + receita_out_1 + receita_ext_1
    receita_bruta_2 = (receita_lp_2 * total_LP_2) + (receita_fr_2 * total_FR_2) + (receita_mult_2 * total_mult_2) + receita_web_2 + receita_out_2 + receita_ext_2
    receita_bruta_3 = (receita_lp_3 * total_LP_3) + (receita_fr_3 * total_FR_3) + (receita_mult_3 * total_mult_3) + receita_web_3 + receita_out_3 + receita_ext_3
    receita_bruta_4 = (receita_lp_4 * total_LP_4) + (receita_fr_4 * total_FR_4) + (receita_mult_4 * total_mult_4) + receita_web_4 + receita_out_4 + receita_ext_4
    receita_bruta_5 = (receita_lp_5 * total_LP_5) + (receita_fr_5 * total_FR_5) + (receita_mult_5 * total_mult_5) + receita_web_5 + receita_out_5 + receita_ext_5

    receita_projetada = pd.DataFrame({"Ano": [ano + 1, ano + 2, ano + 3, ano + 4, ano + 5], "Receita Bruta":[receita_bruta_1, receita_bruta_2, receita_bruta_3, receita_bruta_4, receita_bruta_5]})
    receita_projetada["Receita Liquida"] = receita_projetada["Receita Bruta"] * multiplicador_receita_liq


    ### Cálculo do Lucro Bruto
    ## Markups
    df_cmv = df_dre["CMV"]

    # Markup = razão da receita bruta / CMV
    markup = receita_bruta / (df_cmv * -1)
    var_markup = markup.pct_change()
    cresc_markup = var_markup.mean() + 1

    # Projeta o crescimento do markup com base nos últimos anos (mudança na margem bruta)
    markup_1 = markup[i] * cresc_markup
    markup_2 = markup_1 * cresc_markup
    markup_3 = markup_2 * cresc_markup
    markup_4 = markup_3 * cresc_markup
    markup_5 = markup_4 * cresc_markup

    receita_projetada["Markups"] = [markup_1, markup_2, markup_3, markup_4, markup_5]
    receita_projetada["CMV"] = receita_projetada["Receita Liquida"] / (receita_projetada["Markups"] * -1)

    receita_projetada["Lucro Bruto"] = receita_projetada["Receita Liquida"] + receita_projetada["CMV"]


    ### Projeções de Despesas
    ## Despesas de Lojas próprias e Web Commerce (percentual da receita de lojas proprias + web commerce)

    desp_lojasecom = df_dre.loc[[6,7,8,9,12],"Lojas próprias e Web Commerce"].sum(axis=0)
    particip_desp_lojasecom = desp_lojasecom / (receita_LP.loc[[6,7,8,9,12]] + receita_web.loc[[6,7,8,9,12]]).sum(axis=0)

    ## Projetando Despesas de Lojas próprias e Web Commerce

    desp_lojasecom_1 = (receita_web_1 + receita_lp_1) * particip_desp_lojasecom
    desp_lojasecom_2 = (receita_web_2 + receita_lp_2) * particip_desp_lojasecom
    desp_lojasecom_3 = (receita_web_3 + receita_lp_3) * particip_desp_lojasecom
    desp_lojasecom_4 = (receita_web_4 + receita_lp_4) * particip_desp_lojasecom
    desp_lojasecom_5 = (receita_web_5 + receita_lp_5) * particip_desp_lojasecom

    ## Despesas de venda, logística e suprimentos (percentual da receita total)

    desp_venda = df_dre.loc[[6,7,8,9,12],"Venda, logística e suprimentos"].sum(axis=0)

    particip_desp_venda = desp_venda / df_receita.loc[[6,7,8,9,12],"Receita bruta total"].sum(axis=0)

    ## Projetando Despesas de venda, logística e suprimentos

    desp_venda_1 = receita_bruta_1 * particip_desp_venda
    desp_venda_2 = receita_bruta_2 * particip_desp_venda
    desp_venda_3 = receita_bruta_3 * particip_desp_venda
    desp_venda_4 = receita_bruta_4 * particip_desp_venda
    desp_venda_5 = receita_bruta_5 * particip_desp_venda

    ## G&A (Premissa: Despesas de G&A não tem uma correlação com receita, portanto vão crescer inflação + 5%)
    desp_gea = df_dre.loc[:,"Despesas gerais e administrativas"]
    desp_gea_1 = desp_gea[i] * (cpi_br + 0.05)
    desp_gea_2 = desp_gea_1 * (cpi_br + 0.05)
    desp_gea_3 = desp_gea_2 * (cpi_br + 0.05)
    desp_gea_4 = desp_gea_3 * (cpi_br + 0.05)
    desp_gea_5 = desp_gea_4 * (cpi_br + 0.05)

    ## Outras despesas (média de outras despeas dos anos 2022, 2021, 2018, 2017, 2016)
    desp_outros = df_dre.loc[:, "Outras (despesas) e receitas"]
    media_outros = desp_outros.iloc[[6,7,8,11,12]].mean(axis=0) #2019 outlier


    ## Criando dataframe para armazenar todas as despesas
    despesas = pd.DataFrame({"Despesas LP e Ecom": [desp_lojasecom_1, desp_lojasecom_2, desp_lojasecom_3, desp_lojasecom_4, desp_lojasecom_5],
                             "Despesas Venda e Logística": [desp_venda_1, desp_venda_2, desp_venda_3, desp_venda_4, desp_venda_5 ],
                             "Despesas G&A": [desp_gea_1, desp_gea_2, desp_gea_3, desp_gea_4, desp_gea_5],
                            "Outras despesas": [media_outros, media_outros, media_outros, media_outros, media_outros]})

    ## Cálculo EBITDA (lucro bruto + despeas)

    ebitda =  receita_projetada["Lucro Bruto"] + despesas.sum(axis=1)

    ## Impostos
    particip_impostos = df_dre.loc[[8,9,10,11,12],"Corrente"] / df_dre.loc[[8,9,10,11,12],"EBITDA"]# Participação dos impostos sobre o EBITDA
    media_impostos = particip_impostos.mean()

    impostos = ebitda * media_impostos

    ## CAPEX (percentual do capex sobre o ebitda)
    media_capex = df_capex.loc[[8,9,10,11,12],"CAPEX total"] / df_dre.loc[[8,9,10,11,12],"EBITDA"]
    media_capex = media_capex.mean()

    capex = ebitda * media_capex

    ## Capital de giro
    # Capital de giro calculado com dados extraidos da aba bp
    cg = df_bp.loc[:,["Ativo circulante","Caixa e equivalentes de caixa","Aplicações financeiras", "Passivo circulante","Empréstimos e financiamentos"]]
    cg = (cg["Ativo circulante"] - cg["Caixa e equivalentes de caixa"] - cg["Aplicações financeiras"]) - (cg["Passivo circulante"] - cg["Empréstimos e financiamentos"])
    particip_cg = cg.iloc[[10,11,12]] / df_dre.loc[[10,11,12],"Receita Líquida"]
    particip_cg = particip_cg.mean()

    # Projeção realizada com a participação do capital de giro na receita líquida
    cg_proj = receita_projetada.loc[:, "Receita Liquida"] * particip_cg

    # Cálculo da variação de cg
    var_cg = cg_proj.diff()
    var_cg[0] = cg_proj[1]- cg[i]

    ## FCE
    fce = ebitda - impostos - capex + var_cg

    ## Taxas de desconto

    fator_desconto = [1 + wacc_3, (1 + wacc_3)**2, (1 + wacc_3)**3, (1 + wacc_3)**4, (1 + wacc_3)**5]

    fce_descontado = fce/fator_desconto

    ## Perpetuidade

    g = 0.05

    perpetuidade = fce[4]*((1+g)/(wacc_3-g))
    vp_perpetuidade = perpetuidade / (1 + wacc_3)**5

    ## Preço justo
    caixaliq = df_dre.loc[i,"Caixa líquido³"]
    valor_eco = fce_descontado.sum(axis=0) + vp_perpetuidade + caixaliq
    preco = valor_eco / acoes
    lista_anos.append(ano)
    lista_precos_3.append(preco)

resultado_val_3 = pd.DataFrame({"Ano": lista_anos,"Preço": lista_precos_3})
print(resultado_val_3)

    Ano       Preço
0  2015   81.522456
1  2016   87.668526
2  2017   91.388427
3  2018   95.303360
4  2019   99.604197
5  2020   80.383950
6  2021  133.019731
7  2022  195.779460
